In [3]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm

import pandas as pd
import numpy
import math
import os

import random
import umap.umap_ as umap # need to do pip install umap-learn
from gensim.models import Word2Vec
import networkx as nx

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [4]:
node_df = pd.read_csv("2024_05_05_meta_node_data.csv")
edge_df = pd.read_csv("2024_04_24_meta_edge_sourcetarget_disease_pub_pli_reciprocal.csv")
geneGraph = nx.DiGraph()
%run vizfunctions.ipynb
node_df.set_index('id',inplace=True)
node_df.head()

/var/folders/6q/0qnxsny943385fwzz738241c0000gn/T/ipykernel_9502/4103866634.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  edge_df = pd.read_csv("2024_04_24_meta_edge_sourcetarget_disease_pub_pli_reciprocal.csv")


,symbol,disease_assoc_cat,publication_count,ortholog_count,all_ortho_count
id,,,,,
7414,VCL,disease_assoc,279,5,"{'hsapien': {'entrezid': 7414, 'symbol': 'VCL'..."
4626,MYH8,disease_assoc,36,4,"{'hsapien': {'entrezid': 4626, 'symbol': 'MYH8..."
9722,NOS1AP,disease_assoc,141,5,"{'hsapien': {'entrezid': 9722, 'symbol': 'NOS1..."
9891,NUAK1,non_disease_assoc,70,5,"{'hsapien': {'entrezid': 9891, 'symbol': 'NUAK..."
81788,NUAK2,disease_assoc,46,2,"{'hsapien': {'entrezid': 81788, 'symbol': 'NUA..."


In [3]:
# # Removing all non_omim genes for training/testing
# edge_df = edge_df[
#     (edge_df['source_disease_assoc_cat'] != 'non_omim') &
#     (edge_df['target_disease_assoc_cat'] != 'non_omim')
# ]

In [18]:
# Generating vectors
create_graph(100000)   # 1048576
walks = generate_random_walks(geneGraph, 25, 50, 0.5, 0.5)
str_walks = [[str(n) for n in walk] for walk in walks]

# Removing all non_OMIM nodes
# for key, value in ids_to_vectors.items():
#     if node_df[key]['disease_assoc_cat'] == 'non_omim':
#         vectors.


# int_node_ids = []
# for node_id in node_ids:
#     int_node_ids.append(int(node_id))
# int_node_ids = numpy.array(int_node_ids)

# 10000 -- 1 min 5 sec
# 100000 -- 1124 min 10 sec


In [24]:
model = Word2Vec(str_walks, vector_size = 128, window=5, min_count=0, sg=0, workers=2, hs=0, epochs=1)
node_ids = list(model.wv.index_to_key)  # node ids
vectors = (model.wv.vectors) # vectors
ids_to_vectors = dict(zip(node_ids, list(vectors)))
# dimensionality reduction
umap_embeddings_2d = umap.UMAP(n_neighbors=80, min_dist=0.25, n_components=2, metric='euclidean').fit_transform(vectors)

# Extracting vector and corresponding node
int_node_ids = [int(node_ids[i]) for i in range(len(node_ids))]

In [7]:
disease_association = []
not_disease_associated = []
for index, row in node_df.iterrows():
    if index in geneGraph.nodes:
        if row['disease_assoc_cat'] == 'disease_assoc':
            disease_association.append(row)
        elif row['disease_assoc_cat'] == 'non_disease_assoc':
            not_disease_associated.append(row)

n = len(disease_association)+len(not_disease_associated)
k = len(disease_association)
possibilities = math.comb(n,k)

print("Number of disease associated genes:",len(disease_association))
print("Number of non disease associated genes:", len(not_disease_associated))
print("Chance of randomly labeling correcting:", 1/possibilities)

Number of disease associated genes: 3341
Number of non disease associated genes: 5986
Chance of randomly labeling correcting: 0.0


In [25]:
# Data preprocessing

# Creating a dataset with just the genes, classification, and vectors
vector_data = pd.DataFrame(umap_embeddings_2d)
disease_assoc_cat = numpy.array([node_df.loc[node_id]['disease_assoc_cat'] for node_id in int_node_ids])
publication_count = numpy.array([node_df.loc[node_id]['publication_count'] for node_id in int_node_ids])
ortholog_count = numpy.array([node_df.loc[node_id]['ortholog_count'] for node_id in int_node_ids])
all_ortho_count = numpy.array([node_df.loc[node_id]['all_ortho_count'] for node_id in int_node_ids])

vector_data['gene'] = int_node_ids
# vector_data['publication count'] = publication_count
# vector_data['ortholog_count'] = ortholog_count
# vector_data['all_ortho_count'] = all_ortho_count
vector_data['disease_assoc_cat'] = disease_assoc_cat

vector_data.set_index('gene',inplace=True)

# Creating features
X = vector_data.iloc[:, :-1].values    # Taking vectors as input
y = vector_data['disease_assoc_cat'].values  # Taking disease association as output

# Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [26]:
vector_data.head()

,0,1,disease_assoc_cat
gene,,,
7414,-1.341389,2.148989,disease_assoc
7341,-3.081195,2.787206,disease_assoc
54514,-2.952746,1.726863,non_disease_assoc
1739,-2.755191,2.222238,non_disease_assoc
1742,-2.856904,2.131468,disease_assoc


In [27]:
# Linear SVC
# Initiatlize classifier
svm_model = LinearSVC()

# Perform cross-validation
cv_scores = cross_val_score(svm_model, X_train_scaled, y_train, cv=10, scoring='accuracy')

# Print cross-validation scores
print("Cross-validation accuracies:", cv_scores)
print("Mean cross-validation accuracy:", cv_scores.mean())

# Train classifier
svm_model.fit(X_train_scaled, y_train)

# Evaluating classifier
accuracy = svm_model.score(X_test, y_test)
print(f'Accuracy: {accuracy}')

# Confusion matrix
y_pred = svm_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

#    vector size       edges    nodes       other features           accuracy               (geneGraph, 50, 10, 0.5, 0.5)
#         128         100000   ~9400            none            0.6232114467408585
#         128         100000   ~9400    pub_count, ortho count  0.3783783783783784
#         512         100000   ~9400            none            0.6063492063492063
#         512         100000   ~9400    pub_count, ortho count  0.3904761904761905
#         64          100000   ~9400            none            0.617879746835443
#         64          100000   ~9400    pub_count, ortho count  0.35917721518987344
#         256         100000   ~9400            none            0.6232227488151659
#         32          100000   ~9400            none            0.6195393169181891


#    vector size       edges      nodes       random walk               accuracy               
#         128          100000     ~9400     (50, 10, 0.5, 0.5)       0.6232114467408585
#         256          100000     ~9400     (50, 10, 0.5, 0.5)       0.6232227488151659
#         512          100000     ~9400     (50, 10, 0.5, 0.5)       0.6063492063492063
#         64           100000     ~9400     (50, 10, 0.5, 0.5)       0.617879746835443

#         64           100000     ~9400     (50, 10, 0.8, 0.4)       0.6283676703645008
#         128          100000     ~9400     (50, 10, 0.8, 0.4)       0.6313291139240507
#         256          100000     ~9400     (50, 10, 0.8, 0.4)       0.6313291139240507
#         512          100000     ~9400     (50, 10, 0.8, 0.4)       0.6006314127861089

#         64           100000     ~9400     (50, 10, 0.4, 0.8)       0.6304868316041501
#         128          100000     ~9400     (50, 10, 0.4, 0.8)       0.6282051282051282
#         256          100000     ~9400     (50, 10, 0.4, 0.8)       0.608
#         512          100000     ~9400     (50, 10, 0.4, 0.8)       0.6142172523961661

#         512          100000     ~9400     (50, 10, 1, 0.2)         0.629222309505106

Cross-validation accuracies: [0.59411765 0.59411765 0.59411765 0.59411765 0.59411765 0.59411765
 0.59332024 0.59332024 0.59332024 0.59332024]
Mean cross-validation accuracy: 0.5937986825378483
Accuracy: 0.6059654631083202
[[  0 444   0]
 [  0 772   0]
 [  0  58   0]]


In [28]:
# Kernel SVC

# Initialize classifier
kernelsvc_model = SVC()

# Perform cross-validation
kernel_cv_scores = cross_val_score(kernelsvc_model, X_train_scaled, y_train, cv=10, scoring='accuracy')

# Print cross-validation scores
print("Cross-validation accuracies:", kernel_cv_scores)
print("Mean cross-validation accuracy:", kernel_cv_scores.mean())

# Train classifier
kernelsvc_model.fit(X_train_scaled, y_train)

# Evaluating classifier
accuracy = kernelsvc_model.score(X_test, y_test)
print(f'Accuracy: {accuracy}')

# Confusion matrix
y_pred = kernelsvc_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

Cross-validation accuracies: [0.59411765 0.59411765 0.59411765 0.59411765 0.59411765 0.59411765
 0.59332024 0.59332024 0.59332024 0.59332024]
Mean cross-validation accuracy: 0.5937986825378483
Accuracy: 0.6059654631083202
[[  0 444   0]
 [  0 772   0]
 [  0  58   0]]


In [52]:
# GridSearch CV hyperparameter tuning
# defining parameter range 
param_grid = {'loss':['hinge', 'squared_hinge'],
              'tol': [1, 0.1, 0.01, 0.001, 0.0001],
              'C': [0.1, 1, 10, 100, 1000],
              'max_iter':[1000, 3000, 5000]   
              }  
  
grid = GridSearchCV(svm_model, param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid.fit(X_train, y_train) 

# print best parameter after tuning 
print(grid.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_) 

grid_predictions = grid.predict(X_test) 
  
# print classification report 
print(classification_report(y_test, grid_predictions)) 

Fitting 5 folds for each of 150 candidates, totalling 750 fits
[CV 1/5] END C=0.1, loss=hinge, max_iter=1000, tol=1;, score=0.624 total time=   0.2s
[CV 2/5] END C=0.1, loss=hinge, max_iter=1000, tol=1;, score=0.629 total time=   0.2s
[CV 3/5] END C=0.1, loss=hinge, max_iter=1000, tol=1;, score=0.626 total time=   0.2s
[CV 4/5] END C=0.1, loss=hinge, max_iter=1000, tol=1;, score=0.625 total time=   0.2s
[CV 5/5] END C=0.1, loss=hinge, max_iter=1000, tol=1;, score=0.627 total time=   0.2s
[CV 1/5] END C=0.1, loss=hinge, max_iter=1000, tol=0.1;, score=0.625 total time=   0.4s
[CV 2/5] END C=0.1, loss=hinge, max_iter=1000, tol=0.1;, score=0.629 total time=   0.4s
[CV 3/5] END C=0.1, loss=hinge, max_iter=1000, tol=0.1;, score=0.626 total time=   0.4s
[CV 4/5] END C=0.1, loss=hinge, max_iter=1000, tol=0.1;, score=0.625 total time=   0.5s
[CV 5/5] END C=0.1, loss=hinge, max_iter=1000, tol=0.1;, score=0.626 total time=   0.5s
[CV 1/5] END C=0.1, loss=hinge, max_iter=1000, tol=0.01;, score=0.6

/Users/anusha/Desktop/Neuroinformatics_Internship/HopkinsGeneClassificationProj/env/lib/python3.12/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 1/5] END C=0.1, loss=hinge, max_iter=1000, tol=0.001;, score=0.625 total time=   1.2s
[CV 2/5] END C=0.1, loss=hinge, max_iter=1000, tol=0.001;, score=0.629 total time=   1.4s


/Users/anusha/Desktop/Neuroinformatics_Internship/HopkinsGeneClassificationProj/env/lib/python3.12/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 3/5] END C=0.1, loss=hinge, max_iter=1000, tol=0.001;, score=0.626 total time=   2.2s


...

[CV 5/5] END C=1000, loss=hinge, max_iter=3000, tol=0.01;, score=0.602 total time=  31.0s


/Users/anusha/Desktop/Neuroinformatics_Internship/HopkinsGeneClassificationProj/env/lib/python3.12/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 1/5] END C=1000, loss=hinge, max_iter=3000, tol=0.001;, score=0.589 total time=  28.4s


/Users/anusha/Desktop/Neuroinformatics_Internship/HopkinsGeneClassificationProj/env/lib/python3.12/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 2/5] END C=1000, loss=hinge, max_iter=3000, tol=0.001;, score=0.611 total time=  29.8s
